In [5]:
!wget https://jdbc.postgresql.org/download/postgresql-42.7.4.jar -P /home/jovyan/work

--2024-12-23 18:25:45--  https://jdbc.postgresql.org/download/postgresql-42.7.4.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1086687 (1.0M) [application/java-archive]
Saving to: ‘/home/jovyan/work/postgresql-42.7.4.jar.1’

postgresql-42.7.4.j 100%[===================>]   1.04M  21.3KB/s    in 51s     

2024-12-23 18:26:37 (20.8 KB/s) - ‘/home/jovyan/work/postgresql-42.7.4.jar.1’ saved [1086687/1086687]



In [2]:
from pyspark.sql import SparkSession

def buildSpark():
    spark = SparkSession.builder \
        .master("local") \
        .appName("SparkConnectionTest") \
        .config("spark.jars", "postgresql-42.7.4.jar") \
        .getOrCreate()
    
    #Проверка соединения со Spark
    print("=== Spark Session Info ===")
    print(spark.sparkContext.getConf().getAll())
    return spark

jdbc_url = "jdbc:postgresql://postgres_db:5432/mydatabase"
db_properties = {
    "user": "myuser",
    "password": "mypassword",
    "driver": "org.postgresql.Driver"
}

def runSQL(spark, command):
    try:
        df = spark.read.jdbc(
            url=jdbc_url,
            table=command,  # Тестовый запрос
            properties=db_properties
        )
        print("PostgreSQL Connection Successful!")
        return df
    except Exception as e:
        print("PostgreSQL Connection Failed! If you just downloaded postgree driver, restart docker compose services please.")

# Настройка соединения со Spark Master
spark = buildSpark()
# Проверка соединения с PostgreSQL
runSQL(spark, "(SELECT 1) as test")

spark.stop()

=== Spark Session Info ===
[('spark.master', 'local'), ('spark.app.startTime', '1734981054850'), ('spark.app.submitTime', '1734980303079'), ('spark.driver.port', '40843'), ('spark.executor.id', 'driver'), ('spark.driver.host', 'ee3eb0a478a1'), ('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.se